In [ ]:
%pip install tensorflow

In [3]:
%pip install pandas numpy scikit-learn tensorflow scikeras

#import modules and load the data
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV

url = 'https://github.com/Patrick0481/Intro-to-modeling/raw/refs/heads/main/insurance.csv'
df = pd.read_csv(url)
df.head()

     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.9 kB 682.7 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.9 kB 871.5 kB/s eta 0:00:01
     -------------------------------------- 60.9/60.9 kB 649.9 kB/s eta 0:00:00
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   -- ------------------------------------- 0.6/11.3 MB 19.5 MB/s eta 0:00:01
   ----- ---------------------------------- 1.5/11.3 MB 16.3 MB/s eta 0:00:01
   --------- ------------------------------ 2.6/11.3 MB 18.1 MB/s eta 0:00:01
   ------------ --------------------------- 3.6/11.3 MB 19.0 MB/s eta 0:00:01
   ---------------- ----------------------- 4.7/11.3 MB 20.0 MB/s eta 0:00:01
   -------------------- ------------------- 5.7/11.3 MB 20.3 MB/s eta 0:00:0

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\patri\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\fault_injection\\fault_injection_service_config_parser.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\patri\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'tensorflow.python'

In [7]:
#Prepare the data
df = df.dropna() #remove missings
y = df['charges'] #select Target
X = df.drop('charges',axis=1) #Select features
X = pd.get_dummies(X, drop_first=True) #Turn Fuel_Type into dummy variables
columns = X.columns #create index with column names (needed for last step)
scaler = MinMaxScaler() #initiate the scaler
X = scaler.fit_transform(X) #scale the data
X = pd.DataFrame(X,columns=columns) #turn back into a dataframe

In [8]:
#grid search with Lasso
alpha_range = np.arange(start=1,stop=100,step=0.5) #define a range of 100 alpha values

#define the aspects of the grid search
cv = RepeatedKFold(n_splits=5, random_state=420) #define the cross validation
param = {'alpha':alpha_range} #create a dictionary that contains the range of alpha values
LassoModel = Lasso() #define the model

#create the grid search
LassoM = GridSearchCV(LassoModel, #the lasso model we defined above
                      param_grid=param, #the alpha values we defined above
                      scoring ='neg_mean_absolute_error', #use MAE as measure of fit
                      cv=cv) #the cross validation we defined above
LassoM.fit(X,y)# Fit the random search model
print("Best alpha: ", LassoM.best_params_['alpha']) #print out the best parameter

Best alpha:  82.0


In [9]:
#Exercise 1.1: Ridge grid search
#grid search with Ridge
alpha_range = np.arange(start=1,stop=5,step=0.1) #define a range of 100 alpha values

#define the aspects of the grid search
cv = RepeatedKFold(n_splits=5, random_state=420) #define the cross validation
param = {'alpha':alpha_range} #create a dictionary that contains the range of alpha values
RidgeModel = Ridge() #define the model

#create the grid search
RidgeM = GridSearchCV(RidgeModel, #the lasso model we defined above
                      param_grid=param, #the alpha values we defined above
                      scoring ='neg_mean_absolute_error', #use MAE as measure of fit
                      cv=cv) #the cross validation we defined above
RidgeM.fit(X,y)# Fit the random search model
print("Best alpha: ", RidgeM.best_params_['alpha']) #print out the best parameter

Best alpha:  4.200000000000003


In [10]:
#Exercise 1.2
LassoModel = Lasso(alpha=82)
RidgeModel = Ridge(alpha=4.2)

#Run the cross fold validation again (same as earlier)
cv = RepeatedKFold(n_splits=5, random_state=420)
scoreslasso = cross_val_score(LassoModel, X, y, scoring='neg_mean_absolute_error', cv=cv) 
scoresridge = cross_val_score(RidgeModel, X, y, scoring='neg_mean_absolute_error', cv=cv) 

#Evaluate the model
print('the average prediction error of linear regression was 4210')
lassoMAE = mean(absolute(scoreslasso)) #Calculate the overall mean absolute error
print('the average prediction error with lasso is: %.0f' % lassoMAE) #Print the result
ridgeMAE = mean(absolute(scoresridge)) #Calculate the overall mean absolute error
print('the average prediction error with ridge is: %.0f' % ridgeMAE) #Print the result


the average prediction error of linear regression was 4210
the average prediction error with lasso is: 4155
the average prediction error with ridge is: 4194


In [7]:
#Neural network with cross validation

#First define a function that contains the neural network model
def create_baseline():
    # create model
    nnmodel = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(X.shape[1],)),  # Input layer 
        tf.keras.layers.Dense(64, activation='softplus'), #hidden layer
        tf.keras.layers.Dense(64, activation='softplus'), #hidden layer
        tf.keras.layers.Dense(64, activation='softplus'), #hidden layer
        tf.keras.layers.Dense(1)  # Output layer
    ])
    # Compile model
    nnmodel.compile(loss='mean_absolute_error', #loss is calculated with MAE
                    optimizer='adam', #type of gradient descent
                    metrics=['mae']) #use MAE to evaluate the model
    return nnmodel

# evaluate model 

#define the estimator
estimator = KerasRegressor(build_fn=create_baseline, #you have to use the kerasRegressor wrapper for nn models
                           epochs=100, batch_size=5, #100 epochs and 5 batches
                           verbose=0) #this will shorten the output text
#Define the kfold function
kfold = RepeatedKFold(n_splits=5, n_repeats=1, random_state=42) #I'm using just 5 splits and 1 repeat to save time, but more splits and repeats will be more accurate
#Generate the results
results = cross_val_score(estimator, X, y, cv=kfold,scoring='neg_mean_absolute_error')
#Print the results
print('the average prediction error is: %.0f' % mean(absolute(results)))

c:\Users\Tijmen\anaconda3\envs\env2526\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Tijmen\anaconda3\envs\env2526\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Tijmen\anaconda3\envs\env2526\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Tijmen\anaconda3\envs\env2526\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = sel

the average prediction error is: 3157


In [14]:
#Hyperparameter tuning (warning: takes at least 18 minutes to run)
# Define the function to create the model
def build_model(n_layers=3, n_units=64, activation='softplus'): #start with some inital parameters
    nnmodel = tf.keras.models.Sequential() #define the model
    nnmodel.add(tf.keras.layers.Input(shape=(X.shape[1],)))  # Input layer

    for _ in range(n_layers): #create a for loop that goes through the number of layers, number of units, and activation types
        nnmodel.add(tf.keras.layers.Dense(n_units, activation=activation))
    
    nnmodel.add(tf.keras.layers.Dense(1))  # Output layer
    nnmodel.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae']) #compile the model and use mean absolute error as measure of fit
    return nnmodel

model = KerasRegressor(model=build_model, #for some reason you have to put the parameter distribution in here, otherwise it will give an error
                       n_layers=[1, 2, 3, 4, 5], #Number of hidden layers
                       n_units=[32, 64, 128, 256, 512], #Number of nodes per hidden layer
                       activation=['relu', 'tanh', 'softplus', 'elu'], #activation function
                       epochs=[50, 100, 150, 200], #number of epochs
                       batch_size=[16, 32, 64], #number of batches
                       verbose = 0) #this will shorten the output

param_dist = { #overview of the parameter distribution (this is the same as above)
    'n_layers': [1, 2, 3, 4, 5],
    'n_units': [32, 64, 128, 256, 512],
    'activation': ['relu', 'tanh', 'softplus', 'elu'],
    'epochs': [50, 100, 150, 200],
    'batch_size': [16, 32, 64]
}

random_search = RandomizedSearchCV(estimator=model, #randomly search through the grid
                                   param_distributions=param_dist,
                                    n_iter=10, #number of parameter combinations that are randomly chosen
                                      cv=5, #evaluate the output using 5-fold cross validation 
                                    random_state=42) #set a random seed so that the results will be the same each time

random_search.fit(X, y) #train the models
print("Best parameters found: ", random_search.best_params_) #print the best parameters

Best parameters found:  {'n_units': 512, 'n_layers': 4, 'epochs': 100, 'batch_size': 32, 'activation': 'elu'}


In [15]:
#Exercise 2

#cross validation with the best parameters
def create_baseline():
    # create model
    nnmodel = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(X.shape[1],)),  # Input layer (the shape parts counts how many features you have)
        tf.keras.layers.Dense(512, activation='elu'), #hidden layer
        tf.keras.layers.Dense(512, activation='elu'), #hidden layer
        tf.keras.layers.Dense(512, activation='elu'), #hidden layer
        tf.keras.layers.Dense(512, activation='elu'), #hidden layer
        tf.keras.layers.Dense(1)  # Output layer
    ])
    # Compile model
    nnmodel.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mae']) #compile the model based 
    return nnmodel
# evaluate model  (note if you get an error, upgrade the packages! module not found keras.api_v2 error was because of tensorflow!)
estimator = KerasRegressor(build_fn=create_baseline, epochs=100, batch_size=32,verbose=0) #you have to use the kerasRegressor wrapper for nn models
kfold = RepeatedKFold(n_splits=5, n_repeats = 1) #I'm using just 5 splits and 1 repeat to save time, but more splits and repeats will be more accurate
results = cross_val_score(estimator, X, y, cv=kfold,scoring='neg_mean_absolute_error') #create the results of the cross validation
print('the average prediction error is: %.0f' % mean(absolute(results))) #print the average error

c:\Users\Tijmen\anaconda3\envs\env2526\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Tijmen\anaconda3\envs\env2526\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Tijmen\anaconda3\envs\env2526\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\Tijmen\anaconda3\envs\env2526\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = sel

the average prediction error is: 2085
